# 🐒 Monkey Journey Map

Welcome to monkey adventure visualization! This notebook will create interactive maps showing the monkey's journey, including all activities and health statistics.

## 📦 Setup & Installation

First, let's make sure all required packages are installed. Run the cell below to automatically install any missing packages.

In [13]:
# 📦 Automatic Package Installation
# Run this cell first to install all required packages
import subprocess
import sys

def install_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        return True
    except subprocess.CalledProcessError:
        return False

# List of required packages
required_packages = ["pandas", "folium", "plotly", "numpy"]

print("🔍 Checking for required packages...")

# Check if packages are already installed
missing_packages = []
for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package} - already installed")
    except ImportError:
        missing_packages.append(package)
        print(f"❌ {package} - needs installation")

# Install missing packages
if missing_packages:
    print(f"\n📦 Installing {len(missing_packages)} missing packages...")
    for package in missing_packages:
        print(f"Installing {package}...")
        if install_package(package):
            print(f"✅ {package} installed successfully!")
        else:
            print(f"❌ Failed to install {package}")
    print("\n🎉 Package installation complete!")
else:
    print("\n🎉 All packages are already installed!")

print("\n🚀 Ready to run the notebook! Proceed to the next cell.")

🔍 Checking for required packages...
✅ pandas - already installed
✅ folium - already installed
✅ plotly - already installed
✅ numpy - already installed

🎉 All packages are already installed!

🚀 Ready to run the notebook! Proceed to the next cell.


## Import Required Libraries

First, let's import all the necessary libraries for data handling and map visualization.

In [14]:
import pandas as pd
import folium
from folium import plugins
import json
from datetime import datetime
import numpy as np

# For interactive plotting
import plotly.express as px
import plotly.graph_objects as go

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Load Monkey Journey Data

Let's load the journey data from our JSON file that contains the latest data from the MonkeyMCP service. This includes all the path points, activities, and health statistics from the adventure, generated using our updated 60/40 activity selection algorithm.

The JSON file (`monkey_journey_data.json`) contains fresh data showing:
- **monkey's preferred activities** (60% chance): iPhone Photography, Waterfront Stroll, Adventure Blogging  
- **Common monkey activities** (40% chance): Climbing, Watching, etc.
- Updated health statistics and journey metrics

In [15]:
# Load monkey's Journey Data from JSON file
import json
import os

# Load the journey data from the JSON file
json_file_path = 'monkey_journey_data.json'

# Check if file exists
if os.path.exists(json_file_path):
    with open(json_file_path, 'r') as file:
        monkey_journey = json.load(file)
    print("📊 Journey data loaded successfully from JSON file!")
else:
    print("❌ JSON file not found. Please ensure 'monkey_journey_data.json' exists in the current directory.")
    # Fallback to hardcoded data if file doesn't exist
    monkey_journey = {
        "MonkeyName": "monkey",
        "StartLocation": {"Latitude": 47.608013, "Longitude": -122.335167},
        "StartTime": "2025-07-02T11:35:41.6638431Z",
        "EndTime": "2025-07-02T12:47:41.6638431Z",
        "PathPoints": [],
        "Activities": [],
        "HealthStats": {"Energy": 59, "Happiness": 68, "Hunger": 71, "Social": 76, "Stress": 11, "Health": 87},
        "TotalDuration": "01:12:00",
        "TotalDistanceKm": 0.1946922330484976
    }
    print("📊 Using fallback data.")

print(f"🐒 Monkey: {monkey_journey['MonkeyName']}")
print(f"📍 Path Points: {len(monkey_journey['PathPoints'])}")
print(f"🎯 Activities: {len(monkey_journey['Activities'])}")
print(f"🏃‍♂️ Total Distance: {monkey_journey['TotalDistanceKm']:.3f} km")
print(f"⏱️ Total Duration: {monkey_journey['TotalDuration']}")

# Display the activities with their types (showing the new 60/40 distribution)
print(f"\n🎯 Activities performed:")
activity_types = {}
for activity in monkey_journey['Activities']:
    activity_type = activity['Type']
    if activity_type in activity_types:
        activity_types[activity_type] += 1
    else:
        activity_types[activity_type] = 1
    print(f"  • {activity['Type']}: {activity['Description'][:50]}...")

print(f"\n📊 Activity Type Summary:")
for activity_type, count in activity_types.items():
    print(f"  • {activity_type}: {count} time(s)")

📊 Journey data loaded successfully from JSON file!
🐒 Monkey: Mooch
📍 Path Points: 15
🎯 Activities: 5
🏃‍♂️ Total Distance: 0.461 km
⏱️ Total Duration: 01:36:00

🎯 Activities performed:
  • iPhone Photography: Captured stunning photos with the latest iPhone ca...
  • Waterfront Stroll: Enjoyed a peaceful walk along the Seattle waterfro...
  • Climbing: Climbed a tall tree to get a better view...
  • Climbing: Climbed a tall tree to get a better view...
  • Watching: Sat quietly observing the surrounding environment...

📊 Activity Type Summary:
  • iPhone Photography: 1 time(s)
  • Waterfront Stroll: 1 time(s)
  • Climbing: 2 time(s)
  • Watching: 1 time(s)


## Prepare Geographical Data

Now let's structure the data into pandas DataFrames for easier manipulation and plotting. We'll create separate datasets for the path points and activities.

In [16]:
# Create DataFrame for path points
path_data = []
for i, point in enumerate(monkey_journey['PathPoints']):
    path_data.append({
        'Point': i + 1,
        'Latitude': point['Location']['Latitude'],
        'Longitude': point['Location']['Longitude'],
        'Timestamp': point['Timestamp'],
        'Time': datetime.fromisoformat(point['Timestamp'].replace('Z', '+00:00')).strftime('%H:%M')
    })

path_df = pd.DataFrame(path_data)

# Create DataFrame for activities
activity_data = []
for activity in monkey_journey['Activities']:
    activity_data.append({
        'Type': activity['Type'],
        'Description': activity['Description'],
        'Latitude': activity['Location']['Latitude'],
        'Longitude': activity['Location']['Longitude'],
        'Timestamp': activity['Timestamp'],
        'Time': datetime.fromisoformat(activity['Timestamp'].replace('Z', '+00:00')).strftime('%H:%M'),
        'Duration': activity['Duration'],
        'EnergyChange': activity['EnergyChange']
    })

activity_df = pd.DataFrame(activity_data)

# Display the prepared data
print("🗺️ Path Points DataFrame:")
print(path_df)
print("\n🎯 Activities DataFrame:")
print(activity_df)

🗺️ Path Points DataFrame:
    Point   Latitude   Longitude                     Timestamp   Time
0       1  47.608013 -122.335167  2025-07-02T13:52:08.1079669Z  13:52
1       2  47.607810 -122.335363  2025-07-02T14:04:08.1079669Z  14:04
2       3  47.607804 -122.335363  2025-07-02T14:16:08.1079669Z  14:16
3       4  47.607797 -122.335812  2025-07-02T15:10:08.1079669Z  15:10
4       5  47.607970 -122.335957  2025-07-02T14:32:08.1079669Z  14:32
5       6  47.607635 -122.335648  2025-07-02T15:22:08.1079669Z  15:22
6       7  47.608037 -122.335687  2025-07-02T15:58:08.1079669Z  15:58
7       8  47.608151 -122.335029  2025-07-02T16:08:08.1079669Z  16:08
8       9  47.608688 -122.335183  2025-07-02T15:49:08.1079669Z  15:49
9      10  47.608764 -122.335200  2025-07-02T17:22:08.1079669Z  17:22
10     11  47.608893 -122.335325  2025-07-02T17:43:08.1079669Z  17:43
11     12  47.609125 -122.335193  2025-07-02T16:04:08.1079669Z  16:04
12     13  47.608872 -122.334783  2025-07-02T18:25:08.1079669Z  

## Plot monkey's Journey on a Map

Let's create an interactive map using Folium to visualize monkey's journey. We'll show the path taken and mark all the activity locations.

In [17]:
# Calculate the center point for the map
center_lat = path_df['Latitude'].mean()
center_lon = path_df['Longitude'].mean()

# Create the base map
monkey_map = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=17,
    tiles='OpenStreetMap'
)

# Add the journey path
path_coordinates = list(zip(path_df['Latitude'], path_df['Longitude']))
folium.PolyLine(
    locations=path_coordinates,
    color='blue',
    weight=3,
    opacity=0.8,
    popup='monkey\'s Journey Path'
).add_to(monkey_map)

# Add start point
folium.Marker(
    location=[path_df.iloc[0]['Latitude'], path_df.iloc[0]['Longitude']],
    popup=f"🏁 Journey Start<br>Time: {path_df.iloc[0]['Time']}",
    icon=folium.Icon(color='green', icon='play')
).add_to(monkey_map)

# Add end point
folium.Marker(
    location=[path_df.iloc[-1]['Latitude'], path_df.iloc[-1]['Longitude']],
    popup=f"🏁 Journey End<br>Time: {path_df.iloc[-1]['Time']}",
    icon=folium.Icon(color='red', icon='stop')
).add_to(monkey_map)

# Add path points
for _, point in path_df.iterrows():
    folium.CircleMarker(
        location=[point['Latitude'], point['Longitude']],
        radius=4,
        popup=f"📍 Point {point['Point']}<br>Time: {point['Time']}",
        color='blue',
        fill=True,
        fillColor='lightblue'
    ).add_to(monkey_map)

print("🗺️ Base map with journey path created!")
monkey_map

🗺️ Base map with journey path created!


## Customize Map Visualization

Now let's enhance our map by adding the activity markers with custom icons and colors, plus create an additional visualization showing monkey's health statistics.

In [18]:
# Create enhanced map with activities
enhanced_map = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=17,
    tiles='OpenStreetMap'
)

# Add the journey path
folium.PolyLine(
    locations=path_coordinates,
    color='purple',
    weight=4,
    opacity=0.7,
    popup='🐒 monkey\'s Adventure Path'
).add_to(enhanced_map)

# Define activity icons and colors based on MonkeyLocationService.cs activity types
activity_icons = {
    # Common activities
    'Foraging': {'icon': 'cutlery', 'color': 'orange'},
    'Resting': {'icon': 'bed', 'color': 'blue'},
    'Exploring': {'icon': 'search', 'color': 'purple'},
    'Drinking': {'icon': 'tint', 'color': 'lightblue'},
    'Climbing': {'icon': 'tree', 'color': 'green'},
    'Sunbathing': {'icon': 'sun', 'color': 'yellow'},
    'Playing': {'icon': 'play', 'color': 'pink'},
    'Grooming': {'icon': 'heart', 'color': 'red'},
    'Watching': {'icon': 'eye', 'color': 'gray'},
    'Stretching': {'icon': 'expand', 'color': 'lightgreen'},
    'Snacking': {'icon': 'apple', 'color': 'orange'},
    'Shelter Seeking': {'icon': 'home', 'color': 'brown'},
    'Scent Investigation': {'icon': 'search-plus', 'color': 'darkgreen'},
    'Branch Swinging': {'icon': 'arrows', 'color': 'darkgreen'},
    'Ground Exploration': {'icon': 'map', 'color': 'beige'},
    
    # Baboon specific activities
    'Socializing': {'icon': 'users', 'color': 'blue'},
    
    # Capuchin specific activities
    'Tool Use': {'icon': 'wrench', 'color': 'darkblue'},
    'Fruit Foraging': {'icon': 'apple', 'color': 'red'},
    'Social Learning': {'icon': 'graduation-cap', 'color': 'blue'},
    
    # Blue monkey specific activities
    'Canopy Foraging': {'icon': 'tree', 'color': 'green'},
    'Territory Patrol': {'icon': 'shield', 'color': 'darkred'},
    'Group Bonding': {'icon': 'heart', 'color': 'pink'},
    
    # Squirrel monkey specific activities
    'Acrobatic Leaping': {'icon': 'rocket', 'color': 'orange'},
    'Insect Hunting': {'icon': 'bug', 'color': 'green'},
    'Playful Wrestling': {'icon': 'hand-rock', 'color': 'purple'},
    
    # Golden Lion Tamarin specific activities
    'Mane Grooming': {'icon': 'star', 'color': 'yellow'},
    'Tree Hole Foraging': {'icon': 'circle', 'color': 'brown'},
    'Family Bonding': {'icon': 'heart', 'color': 'red'},
    
    # Howler monkey specific activities
    'Howling': {'icon': 'volume-up', 'color': 'darkred'},
    'Leaf Eating': {'icon': 'leaf', 'color': 'green'},
    'Territory Marking': {'icon': 'flag', 'color': 'red'},
    
    # Japanese Macaque specific activities
    'Hot Spring Bath': {'icon': 'bath', 'color': 'lightblue'},
    'Snow Play': {'icon': 'snowflake', 'color': 'white'},
    'Potato Washing': {'icon': 'tint', 'color': 'blue'},
    
    # Mandrill specific activities
    'Colorful Display': {'icon': 'star', 'color': 'rainbow'},
    'Ground Foraging': {'icon': 'search', 'color': 'brown'},
    'Troop Leadership': {'icon': 'crown', 'color': 'gold'},
    
    # Proboscis monkey specific activities
    'Swimming': {'icon': 'swimmer', 'color': 'lightblue'},
    'Nose Flexing': {'icon': 'smile', 'color': 'orange'},
    'Mangrove Foraging': {'icon': 'tree', 'color': 'darkgreen'},
    
    # Red-shanked douc specific activities
    'Colorful Posing': {'icon': 'camera', 'color': 'red'},
    'Leaf Selection': {'icon': 'leaf', 'color': 'green'},
    'Arboreal Movement': {'icon': 'arrows', 'color': 'green'},
    
    # Sebastian (urban monkey) specific activities
    'Tech Shopping': {'icon': 'shopping-cart', 'color': 'blue'},
    'Coffee Break': {'icon': 'coffee', 'color': 'brown'},
    'Social Media': {'icon': 'share', 'color': 'blue'},
    
    # Henry (desert urban monkey) specific activities
    'iOS Testing': {'icon': 'mobile', 'color': 'gray'},
    'Desert Exploration': {'icon': 'sun', 'color': 'orange'},
    'Travel Planning': {'icon': 'plane', 'color': 'blue'},
    
    # monkey (urban waterfront monkey) specific activities
    'iPhone Photography': {'icon': 'camera', 'color': 'black'},
    'Waterfront Stroll': {'icon': 'ship', 'color': 'blue'},
    'Adventure Blogging': {'icon': 'edit', 'color': 'green'},
    
    # Default activities
    'Banana Finding': {'icon': 'apple', 'color': 'yellow'},
    'Swinging': {'icon': 'arrows', 'color': 'green'},
    'Mutual Grooming': {'icon': 'heart', 'color': 'pink'},
    
    # Fallback for unknown activities
    'Unknown': {'icon': 'question', 'color': 'gray'}
}

# Add activity markers
for _, activity in activity_df.iterrows():
    icon_info = activity_icons.get(activity['Type'], activity_icons['Unknown'])
    
    folium.Marker(
        location=[activity['Latitude'], activity['Longitude']],
        popup=f"""
        <b>🎯 {activity['Type']}</b><br>
        📝 {activity['Description']}<br>
        ⏰ Time: {activity['Time']}<br>
        ⏳ Duration: {activity['Duration']}<br>
        ⚡ Energy Change: {'+' if activity['EnergyChange'] >= 0 else ''}{activity['EnergyChange']}
        """,
        icon=folium.Icon(color=icon_info['color'], icon=icon_info['icon'])
    ).add_to(enhanced_map)

# Add start and end markers
folium.Marker(
    location=[path_df.iloc[0]['Latitude'], path_df.iloc[0]['Longitude']],
    popup="🏁 monkey's Journey Begins!",
    icon=folium.Icon(color='green', icon='play', prefix='fa')
).add_to(enhanced_map)

folium.Marker(
    location=[path_df.iloc[-1]['Latitude'], path_df.iloc[-1]['Longitude']],
    popup="🏁 Journey Complete!",
    icon=folium.Icon(color='red', icon='stop', prefix='fa')
).add_to(enhanced_map)

print("🎨 Enhanced map with comprehensive activity icons created!")
print(f"📊 Total activity types supported: {len(activity_icons)}")
enhanced_map

🎨 Enhanced map with comprehensive activity icons created!
📊 Total activity types supported: 56


## monkey's Health Dashboard

Let's create some visualizations to show monkey's health statistics and journey summary.

In [19]:
# Health Statistics
health_stats = monkey_journey['HealthStats']

# Create health stats bar chart
health_df = pd.DataFrame(list(health_stats.items()), columns=['Metric', 'Value'])

fig_health = px.bar(
    health_df, 
    x='Metric', 
    y='Value',
    title="🐒 monkey's Health Statistics",
    color='Value',
    color_continuous_scale='Viridis',
    text='Value'
)

fig_health.update_traces(texttemplate='%{text}', textposition='outside')
fig_health.update_layout(
    xaxis_title="Health Metrics",
    yaxis_title="Score (0-100)",
    showlegend=False
)

fig_health.show()

# Activity summary
print("📊 Journey Summary:")
print(f"🐒 Monkey: {monkey_journey['MonkeyName']}")
print(f"⏱️ Total Duration: {monkey_journey['TotalDuration']}")
print(f"📏 Total Distance: {monkey_journey['TotalDistanceKm']:.3f} km")
print(f"🎯 Activities Completed: {len(monkey_journey['Activities'])}")
print(f"📍 Path Points Recorded: {len(monkey_journey['PathPoints'])}")

print("\n🎯 Activity Breakdown:")
for activity in monkey_journey['Activities']:
    print(f"  • {activity['Type']}: {activity['Duration']} (+{activity['EnergyChange']} energy)")

print(f"\n💪 Final Health Score: {health_stats['Health']}/100")
print(f"😊 Happiness Level: {health_stats['Happiness']}/100")
print(f"😌 Stress Level: {health_stats['Stress']}/100 (Lower is better!)")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed